In [1]:
import pandas as pd
import jsonlines
import os
from glob import glob

/home/heegyu/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
files = glob("../outputs/*/*/prometheus-13b-v1.0/*.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            return int(score)
        else:
            return -1
    else:
        return -1

for file in files:
    df = pd.read_json(file, lines=True)
    df["score"] = df.judge.apply(filter_score)
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(16905, 7)

In [3]:
df["output_len"] = df.output.apply(len)
df.head()
# df[df.score < 0].judge.apply(lambda x: x['critic']).tolist()


,instruction,output,generator,dataset,judge,judge_model,score,output_len
0,What are the names of some famous actors that ...,Some famous actors that started their careers ...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,542
1,How did US states get their names?,The names of many US states are derived from E...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,933
2,"Hi, my sister and her girlfriends want me to p...",Sure! Here's a step-by-step breakdown of kickb...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,1780
3,What is some cool music from the 1920s?,Cool music from the 1920s may include:\n\n1. T...,heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,1,969
4,How do I wrap a present neatly?,"Yes, here are some tips to help you wrap a pre...",heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epo...,helpful_base,"{'critic': ' Based on the scoring rubric, the ...",kaist-ai/prometheus-13b-v1.0,2,998


In [4]:
df[df.score > 0].groupby("generator").agg({
    "score": ["mean", "median", "std", "count"],
})

score                   \
                                                        mean median       std   
generator                                                                       
heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.784826    1.0  1.311780   
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.520548    1.0  1.068951   
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1.436567    1.0  1.020198   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1  1.829390    1.0  1.296276   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2  1.737888    1.0  1.256671   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3  1.603727    1.0  1.177414   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.517413    1.0  1.078925   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.087065    1.0  0.478563   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.072050    1.0  0.431677   
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1  1.383085    1.0  0.976696   
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2  1.689055    1.0  1.304396   
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3  1.888060    1.0  1.425583   
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1  1.735990    1.0  1.329706   
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2  1.660848    1.0  1.241073   
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3  1.479452    1.0  1.101128   
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1.618634    1.0  1.170704   
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1.513682    1.0  1.085882   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.869240    1.0  1.373634   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.901863    1.0  1.389928   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.776119    1.0  1.288759   
text_davinci_003                                    1.778882    1.0  1.263062   

                                                          
                                                   count  
generator                                                 
heegyu/TinyLlama-1.1b-feedback-all@epoch-1           804  
heegyu/TinyLlama-1.1b-feedback-all@epoch-2           803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-3           804  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1   803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   804  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   804  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   805  
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1   804  
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2   804  
heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3   804  
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1   803  
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2   802  
heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3   803  
heegyu/TinyLlama-1.1b-max-margin@epoch-2             805  
heegyu/TinyLlama-1.1b-max-margin@epoch-3             804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   803  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   805  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   804  
text_davinci_003                                     805

In [44]:
df[df.score > 0].groupby("generator").agg({
    "score": "mean",
    "output_len": ["mean", "median", "std", "count"],
})

score   output_len  \
                                                        mean         mean   
generator                                                                   
heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.784826  1156.945274   
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.520548  1336.073474   
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1.436567  1274.685323   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1  1.829390  1035.331258   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2  1.737888   981.081988   
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3  1.603727  1107.804969   
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...  1.558739  1086.747851   
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1.618634  1237.853416   
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1.513682  1294.924129   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.869240  1041.265255   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.901863  1040.172671   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.776119  1133.180348   
text_davinci_003                                    1.778882  1059.672050   

                                                                              
                                                    median         std count  
generator                                                                     
heegyu/TinyLlama-1.1b-feedback-all@epoch-1           990.5  901.824627   804  
heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1195.0  918.657444   803  
heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1136.0  909.712958   804  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1   893.0  800.633616   803  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2   797.0  793.368868   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3   918.0  879.540126   805  
heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di...   920.0  853.469184   349  
heegyu/TinyLlama-1.1b-max-margin@epoch-2            1107.0  866.385656   805  
heegyu/TinyLlama-1.1b-max-margin@epoch-3            1169.5  889.523366   804  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   892.0  825.778962   803  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   897.0  822.370682   805  
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...   987.0  878.581902   804  
text_davinci_003                                     956.0  783.421212   805

# PairRM

In [3]:
files = glob("../outputs/*/*/pairrm/*.json")
dfs = []

for file in files:
    df = pd.read_json(file, lines=True)
    df["dataset"] = os.path.basename(file).split(".")[0]
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(9780, 6)

In [7]:
df.groupby(["dataset", "generator"]).agg({
    "judge": ["mean", "median", "std", "count"],
})

judge  \
                                                                   mean   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.149573   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.218166   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.611811   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -3.949996   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.393866   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.331274   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.657126   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -5.949652   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.698814   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.780337   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.670845   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.336723   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.290145   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.174094   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.410455   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.185481   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.247956   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.745288   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.755498   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.836002   
           text_davinci_003                                   -3.869928   

                                                                         \
                                                                 median   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.185547   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.335938   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.738281   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -4.121094   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.478516   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.300781   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.960938   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -6.359375   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.826172   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.875000   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.857422   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.525391   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.414062   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.203125   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.480469   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.343750   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.367188   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.880859   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.910156   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.935547   
           text_davinci_003                                   -4.000000   

                                                                               
                                                                    std count  
dataset    generator                                                           
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.350452   276  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.435997   361  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1